In [168]:
import numpy as np
import pandas as pd
import pydicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib import patches
import SimpleITK as sitk
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from pathlib import Path
import json
from preproc import load_scan, resample
from tqdm.notebook import tqdm

ROOT_ISO = Path('/scratch/zc2357/cv/final/datasets/luna16_iso')

with open(ROOT_ISO / 'uid_to_subset.json') as f:
    uid_to_subset = json.load(f)

candidates = pd.read_csv(ROOT_ISO / 'candidates_V2.csv').set_index('seriesuid')
metadata = pd.read_csv(ROOT_ISO / 'seriesuid_isometric_spacing_origin.csv').set_index('seriesuid')
metadata['directionX'] = 100
metadata['directionY'] = 100
metadata['directionZ'] = 100

ROOT_RAW = Path('/scratch/zc2357/cv/final/datasets/luna16')

for seriesuid, subset in tqdm(uid_to_subset.items()):
    img, arr = load_scan((ROOT_RAW / subset / (seriesuid + '.mhd')).as_posix())
    direction = np.array(img.GetDirection()).round(5)
    directionX, directionY, directionZ = direction[[0, 4, 8]]  # assuming diagonal
    metadata.loc[seriesuid, 'directionX'] = directionX
    metadata.loc[seriesuid, 'directionY'] = directionY
    metadata.loc[seriesuid, 'directionZ'] = directionZ

metadata.reset_index().to_csv(ROOT_ISO / 'seriesuid_isometric_spacing_origin_direction.csv', index=False)
df = pd.read_csv(ROOT_ISO / 'seriesuid_isometric_spacing_origin_direction.csv')